In [1]:
import numpy as np 
import tensorflow as tf

In [2]:
# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.7.0


In [2]:
from ModelBuilder import ModelBuilder
from DataReader import DataReader

In [20]:
def custom_loss(N_R: float, N_D: float):
    '''permette di avere parametri nella loss function'''
    
    def Loss(yTrue, yPred):
        return tf.reduce_sum(-1.*yTrue*(yPred) + (1-yTrue)*N_D/float(N_R)*(tf.exp(yPred)-1))
    
    return Loss

In [3]:
N_Ref = 200000
N_Bkg = 3000
N_R = N_Ref
N_D = N_Bkg

input_size = 2
output_size = 1
latent_size = 4
n_layers = 3
layers = [input_size]

for _ in range(n_layers):
    layers.append(latent_size)
layers.append(output_size)

print(layers)
hidden_layers = layers[1:-1]

[2, 4, 4, 4, 1]


In [4]:
DATA_PATH = "../../data/RUN001231_channels.h5"
reader = DataReader(filename=DATA_PATH)
ref_df  = reader.build_sample(ndata=N_Ref)
data_df = reader.build_sample(ndata=N_Bkg)

In [21]:
targetData  = np.ones_like(data_df)
targetRef   = np.zeros_like(ref_df)
target      = np.concatenate((targetData, targetRef), axis=0)
feature     = np.concatenate((data_df, ref_df), axis=0)

In [22]:
batch_size = feature.shape[0]
n_inputs = feature.shape[1]

In [41]:
NPLModel = ModelBuilder(
                input_shape=(None, input_size),
                architecture=layers,
                weight_clipping=7,
                activation='tanh',        # usare tanh, sigmoid non va molto bene
                name="test"                     # parametro della custom activation function 
            )

In [42]:
NPLModel.compile(loss=custom_loss(N_R, N_D), optimizer="adam")

In [43]:
hist = NPLModel.fit(feature, target, batch_size=batch_size, epochs=10000, verbose=True)


Epoch 1/10000


NotImplementedError: in user code:

    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/niklai/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 475, in call
        ra